## tesing dataloader V5

new ideas with ordinal c and yc 
has ordinal binary and continous outputs

In [2]:
# Load dependencies
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split



from utils.graph import *
from utils.loss_ordinal import *
from utils.tram_model_helpers import *
from utils.tram_models import *
from utils.tram_data import *


import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Train with GPU support.")
else:
    device = torch.device('cpu')
    print("No GPU found, train with CPU support.")

Train with GPU support.


adjustet funcitnos for ordinal outcomes 

dev ordinal

In [3]:
experiment_name = "testing_v5_dataloader"   ## <--- set experiment name
seed=42
np.random.seed(seed)

LOG_DIR="/home/bule/TramDag/dev_experiment_logs"
EXPERIMENT_DIR = os.path.join(LOG_DIR, experiment_name)
DATA_PATH = EXPERIMENT_DIR # <----------- change to different source if needed
CONF_DICT_PATH = os.path.join(EXPERIMENT_DIR, f"configuration.json")

In [4]:
df = pd.DataFrame({
    "x1": np.random.normal(loc=0, scale=1, size=1000),
    # "x2": np.random.uniform(low=0, high=10, size=1000),
    "ord_bin1":  np.random.binomial(1, p=0.4, size=1000),
    "ord_multi": np.random.choice([0, 1, 2, 3], size=1000, p=[0.2, 0.3, 0.3, 0.2]),
    "ord_bin2":  np.random.binomial(1, p=0.4, size=1000),
})

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

quantiles = train_df.quantile([0.05, 0.95])
min_vals = quantiles.loc[0.05]
max_vals = quantiles.loc[0.95]

In [5]:
### New Functions and classes


def create_levels_dict(df:pd.DataFrame,data_type:dict):
    # creates the levels dictionary for variables which should be modelled ordinaly 
    levels_dict={}
    for variable,datatype in data_type.items():
            if "ordinal" in datatype.lower():
                unique_vals = set(df[variable].dropna().unique())
                num_classes = len(unique_vals)

                expected_vals = set(range(num_classes))
                if unique_vals != expected_vals:
                    raise ValueError(
                        f"Variable '{variable}' has values {sorted(unique_vals)}, "
                        f"but expected values are {sorted(expected_vals)} (0 to {num_classes - 1}). "
                        "Multiclass ordinal variables must be zero-indexed and contiguous."
                )
                levels_dict[variable]=len(np.unique(df[variable]))
    return levels_dict   




def create_node_dict_v3(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict=None):
    """
    Creates a configuration dictionary for TRAMADAG based on an adjacency matrix,
    a neural network names matrix, and a data type dictionary.
    """
    if not validate_adj_matrix(adj_matrix):
        raise ValueError("Invalid adjacency matrix. Please check the criteria.")
    
    if len(data_type) != adj_matrix.shape[0]:
        raise ValueError("Data type dictionary should have the same length as the adjacency matrix.")
    
    target_nodes = {}
    G, edge_labels = create_nx_graph(adj_matrix, node_labels=list(data_type.keys()))
    
    sources = [node for node in G.nodes if G.in_degree(node) == 0]
    sinks = [node for node in G.nodes if G.out_degree(node) == 0]
    
    for i, node in enumerate(G.nodes):
        parents = list(G.predecessors(node))
        target_nodes[node] = {}
        target_nodes[node]['Modelnr'] = i
        target_nodes[node]['data_type'] = data_type[node]
        
        # write the levels of the ordinal outcome
        if 'ordinal' in data_type[node]:
            if levels_dict is None:
                raise ValueError(
                    "levels_dict must be provided for ordinal nodes; "
                    "e.g. levels_dict={'x3': 3}"
                )
            if node not in levels_dict:
                raise KeyError(
                    f"levels_dict is missing an entry for node '{node}'. "
                    f"Expected something like levels_dict['{node}'] = <num_levels>"
                )
            target_nodes[node]['levels'] = levels_dict[node]
    
        target_nodes[node]['node_type'] = "source" if node in sources else "sink" if node in sinks else "internal"
        target_nodes[node]['parents'] = parents
        target_nodes[node]['parents_datatype'] = {parent:data_type[parent] for parent in parents}
        target_nodes[node]['transformation_terms_in_h()'] = {parent: edge_labels[(parent, node)] for parent in parents if (parent, node) in edge_labels}
        target_nodes[node]['min'] = min_vals.iloc[i].tolist()   
        target_nodes[node]['max'] = max_vals.iloc[i].tolist()

        
        transformation_term_nn_models = {}
        for parent in parents:
            parent_idx = list(data_type.keys()).index(parent)  
            child_idx = list(data_type.keys()).index(node) 
            
            if nn_names_matrix[parent_idx, child_idx] != "0":
                transformation_term_nn_models[parent] = nn_names_matrix[parent_idx, child_idx]
        target_nodes[node]['transformation_term_nn_models_in_h()'] = transformation_term_nn_models
    return target_nodes



In [6]:

# datatype= "ordinal_Xc_Yc" # as X its continous aswell as Y continous
# datatype= "ordinal_Xc_Yo"
# datatype= "ordinal_Xn_Yc"
# datatype= "ordinal_Xn_Yo"


In [7]:
def preprocess_inputs_v2(x, transformation_terms, device='cuda'):
    """
    Prepares model input by grouping features by transformation term base:
      - ci11, ci12 → 'ci1' (intercept)
      - cs11, cs12 → 'cs1' (shift)
      - cs21 → 'cs2' (another shift group)
      - cs, ls → treated as full group keys
    Returns:
      - int_inputs: Tensor of shape (B, n_features) for intercept model
      - shift_list: List of tensors for each shift model, shape (B, group_features)
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transformation_terms=list(transformation_terms)
    
    ## if there is only a source so transforamtion terms is 0:
    x = [xi.to(device, non_blocking=True) for xi in x]
    if len(transformation_terms)== 0:
        x = [xi.unsqueeze(1) for xi in x] 
        int_inputs= x[0]
        return int_inputs, None

    # Always ensure there's an intercept term
    if not any('ci' in str(value) for value in transformation_terms):
        transformation_terms.insert(0, 'si')

    # Lists to collect intercept tensors and shift‐groups
    int_tensors = []
    shift_groups = []

    # Helpers to track the “current” shift‐group for numbered suffixes
    current_group = None
    current_key = None

    for tensor, term in zip(x, transformation_terms):
        # 1) INTERCEPT terms (si*, ci*)
        if term.startswith(('si','ci')):
            int_tensors.append(tensor)

        # 2) SHIFT terms (cs*, ls*)
        elif term.startswith(('cs','ls')):
            # numbered suffix → group by the first 3 chars (e.g. 'cs11'/'cs12' → 'cs1')
            if len(term) > 2 and term[2].isdigit():
                key = term[:3]
                # start a new group if key changed
                if current_group is None or current_key != key:
                    current_group = []
                    shift_groups.append(current_group)
                    current_key = key
                current_group.append(tensor)

            # lone 'cs' or 'ls' → always its own group
            else:
                current_group = [tensor]
                shift_groups.append(current_group)
                current_key = None
        else:
            raise ValueError(f"Unknown transformation term: {term}")

    # Intercept: should be exactly one group
    if len(int_tensors) == 0:
        raise ValueError("No intercept tensors found!")
    int_inputs = torch.cat(
        [t.to(device, non_blocking=True).view(t.shape[0], -1) for t in int_tensors],
        dim=1
    )

    shift_list = [
        torch.cat([t.to(device, non_blocking=True).view(t.shape[0], -1) for t in group], dim=1)
        for group in shift_groups
    ]

    return int_inputs, shift_list if shift_list else None

# Generic DATAset V5

In [8]:
class GenericDataset_v5(Dataset):
    def __init__(
        self,
        df,
        target_col,
        target_nodes=None,
        parents_dataype_dict=None,
        transformation_terms_in_h=None,
        return_intercept_shift=True,
        transform=None
    ):
        """
        df: pd.DataFrame
        target_col: str
        target_nodes: dict mapping each node → metadata (including 'data_type')
        parents_dataype_dict: dict var_name → "cont"|"ord"|"other"
        transform: torchvision transform for images
        transformation_terms_in_h: dict for intercept logic
        """
        self.return_intercept_shift=return_intercept_shift
        self.df = df.reset_index(drop=True)
        self.target_col = target_col
        self.target_nodes = target_nodes or {}
        self.parents_dataype_dict = parents_dataype_dict or {}
        self.predictors = list(self.parents_dataype_dict.keys())
        self.transform = transform
        self.transformation_terms_in_h = transformation_terms_in_h or {}
        
        self.target_is_source= True if target_nodes[self.target_col].get('node_type').lower() == "source" else False
        self.h_needs_simple_intercept=True if all('i' not in str(v) for v in self.transformation_terms_in_h.values()) else False
        self.ordinal_num_classes ={
                        var: self.df[var].nunique() for var in self.predictors
                        if "ordinal" in self.parents_dataype_dict[var].lower() and "Xn".lower() in self.parents_dataype_dict[var].lower()
                        }
        
        self.target_data_type=self.target_nodes[self.target_col].get('data_type').lower()
        self.target_num_classes=self.target_nodes[self.target_col].get('levels') or None # should be none anywasy if levels not exits
        
        self._intercept_indices, self._shift_group_indices = self._group_transform_terms(
            list(self.transformation_terms_in_h.values())
        )
        
        
        #checks
        self._check_multiclass_predictors_of_df()
        self._check_ordinal_levels()

    def _group_transform_terms(self, transformation_terms):
        intercept_indices = []
        shift_group_indices = []

        current_key = None
        current_group = []

        for idx, term in enumerate(transformation_terms):
            if term.startswith(('si', 'ci')):
                intercept_indices.append(idx)
            elif term.startswith(('cs', 'ls')):
                if len(term) > 2 and term[2].isdigit():
                    key = term[:3]
                    if current_key != key:
                        if current_group:
                            shift_group_indices.append(current_group)
                        current_group = [idx]
                        current_key = key
                    else:
                        current_group.append(idx)
                else:
                    if current_group:
                        shift_group_indices.append(current_group)
                    shift_group_indices.append([idx])
                    current_group = []
                    current_key = None
            else:
                raise ValueError(f"Unknown transformation term: {term}")

        if current_group:
            shift_group_indices.append(current_group)

        return intercept_indices, shift_group_indices
    
    def _preprocess_inputs(self, x_data):
        if len(self._intercept_indices) == 0:
            return x_data[0].unsqueeze(0).unsqueeze(0), []  # Ensure 2D: shape (1, 1)

        # Intercepts
        int_tensors = [x_data[i] for i in self._intercept_indices]
        int_inputs = torch.cat([
            t.view(1, -1) if t.dim() <= 1 else t.view(1, -1)
            for t in int_tensors
        ], dim=1)

        # Shift groups
        shift_list = []
        for group in self._shift_group_indices:
            group_tensors = [x_data[i] for i in group]
            group_tensor = torch.cat([
                t.view(1, -1) if t.dim() <= 1 else t.view(1, -1)
                for t in group_tensors
            ], dim=1)
            shift_list.append(group_tensor)

        return int_inputs, shift_list

        
    
    def _transform_y(self,row):
        #returns continous or onehot encoded target
        if  self.target_data_type=="continous" or "Yc".lower() in self.target_data_type:
                y = torch.tensor(row[self.target_col], dtype=torch.float32)
                return y
        elif self.target_num_classes is not None:
                        raw = row[self.target_col]
                        y_int = int(raw)
                        y = F.one_hot(torch.tensor(y_int, dtype=torch.long), num_classes=self.target_num_classes).float().squeeze()
                        return y    
        else:
            raise ValueError(
                f"Could not determine how to encode target '{self.target_col}'.\n"
                f"target_data_type: {self.target_data_type}, target_num_classes: {self.target_num_classes}"
            )

        
    def _check_multiclass_predictors_of_df(self):
        # checks whether the predictors(predictors) have star
        for var in self.predictors:
            dtype = self.parents_dataype_dict[var]
            # continous predictors or ordinal predictors modelled as continous are skippped
            if "ordinal" not in dtype:
                continue
            if "Xc" in dtype:
                continue
            
            # if ordinal varibale is nominally modelled -> check wheter the levels of the varibales start at 0 , minimal level must be 0
            elif "Xn".lower() in dtype.lower():
                unique_vals = set(self.df[var].dropna().unique())
                num_classes = len(unique_vals)

                expected_vals = set(range(num_classes))
                if unique_vals != expected_vals:
                    raise ValueError(
                        f"Variable '{var}' has values {sorted(unique_vals)}, "
                        f"but expected values are {sorted(expected_vals)} (0 to {num_classes - 1}). "
                        "Multiclass ordinal predictors must be zero-indexed and contiguous."
                )
            else:
                continue
            
    def _check_ordinal_levels(self):
        """
        Ensures all ordinal variables (including the target) have a 'levels' key in target_nodes,
        and that the values in the DataFrame are zero-indexed and contiguous.
        """
        all_ordinal_vars = [self.target_col] if "ordinal" in self.target_nodes[self.target_col]['data_type'] else []

        all_ordinal_vars+=[
            var for var in self.predictors
            if "ordinal" in self.parents_dataype_dict[var].lower()
            and "xn" in self.parents_dataype_dict[var].lower()
        ]
        
        for var in all_ordinal_vars:
            node_info = self.target_nodes.get(var, {})
            levels = node_info.get("levels")
            if levels is None:
                raise ValueError(
                    f"[Ordinal Check] Variable '{var}' is marked ordinal but has no 'levels' entry in target_nodes."
                )

            unique_vals = sorted(self.df[var].dropna().unique())
            expected_vals = list(range(levels))

            if unique_vals != expected_vals:
                raise ValueError(
                    f"[Ordinal Check] Variable '{var}' has values {unique_vals}, "
                    f"but expected values {expected_vals} (0 to {levels - 1}).\n"
                    f"Fix this in the DataFrame or correct 'levels' in target_nodes."
                )
                
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # ---------------------------------------- SOURCE NODE: no parents → x = [1.0] ---
        if  self.target_is_source:
            x_data = [torch.tensor(1.0)]
            y=self._transform_y(row)
            
            if self.return_intercept_shift:
                shift_list=[]
                return (x_data, shift_list), y
            else:    
                return tuple(x_data), y

        # ---------------------------------------- if not source prepare data  ---
        x_data = []

        # --- SIMPLE INTERCEPT if needed  first term in x is x = [1.0]---
        if self.h_needs_simple_intercept:
            x_data.append(torch.tensor(1.0))

        # --- BUILD FEATURES ---
        for var in self.predictors:
            dtype = self.parents_dataype_dict[var].lower()
            ## Continous  feature
            if dtype == "continous" or "Xc".lower() in dtype:
                x_data.append(torch.tensor(row[var], dtype=torch.float32))
                
            ## Ordinal feature , if it has more thatn 2 classes it uses onehotencodig, if binary use just 0 and 1
            elif "ordinal" in dtype and "Xn".lower() in dtype:
                x_ord = int(row[var])
                var_num_classes = self.ordinal_num_classes[var]
                x_ord_onehot = F.one_hot(torch.tensor(x_ord, dtype=torch.long),num_classes=var_num_classes).float()
                
                x_data.append(x_ord_onehot.squeeze())

            else:  # "other"
                img = Image.open(row[var]).convert("RGB")
                if self.transform:
                    img = self.transform(img)
                x_data.append(img)

        # --- BUILD TARGET ---
        y=self._transform_y(row)
            
        
        if self.return_intercept_shift:
            int_input, shift_list = self._preprocess_inputs(
                [xi.unsqueeze(0) if xi.dim() == 1 else xi for xi in x_data]
            )
            if shift_list is None:
                shift_list = []  
            
            return (int_input, shift_list), y
        
        else:    
            return tuple(x_data), y

def get_dataloader_v5(node, target_nodes, train_df, val_df, batch_size=32,return_intercept_shift=False, verbose=False):
    
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    ordered_parents_dataype_dict, ordered_transformation_terms_in_h, _ = ordered_parents(node, target_nodes)
    if verbose:
        print(f"Parents dtype: {ordered_parents_dataype_dict}")
    train_ds = GenericDataset_v5(train_df,target_col=node,target_nodes=target_nodes,parents_dataype_dict=ordered_parents_dataype_dict,transform=transform,transformation_terms_in_h=ordered_transformation_terms_in_h,return_intercept_shift=return_intercept_shift)
    val_ds = GenericDataset_v5(val_df,target_col=node,target_nodes=target_nodes,parents_dataype_dict=ordered_parents_dataype_dict,transform=transform,transformation_terms_in_h=ordered_transformation_terms_in_h,return_intercept_shift=return_intercept_shift)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    return train_loader, val_loader

tzhis version only works for the flag ,return_intercept_shift=False and preprocessing is done outside the class

In [9]:



data_type={key:value for key, value in zip(train_df.columns, ['continous']+['ordinal_Xn_Yo']+['ordinal_Xn_Yo']+['ordinal_Xn_Yo'])}
print(data_type)
configuration_dict=new_conf_dict(experiment_name,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)

adj_matrix=np.array([['0', '0', 'cs11', 'ci11'],
                     ['0', '0', 'cs12', 'ci12'],
                     ['0', '0', '0', 'cs'],
                     ['0', '0', '0', '0']])
            
nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
levels_dict=create_levels_dict(df,data_type)
target_nodes=create_node_dict_v3(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict)

for node in target_nodes:
    print(f'-----------------{node}-------------------')
    train_loader, _ = get_dataloader_v5(node, target_nodes, train_df, val_df, batch_size=1,return_intercept_shift=False, verbose=False)
    # (int_input, shift_list), y =next(iter(train_loader))
    x, y =next(iter(train_loader))


    print(x)
    print(y)


{'x1': 'continous', 'ord_bin1': 'ordinal_Xn_Yo', 'ord_multi': 'ordinal_Xn_Yo', 'ord_bin2': 'ordinal_Xn_Yo'}
*************
 Model has Complex intercepts and Complex shifts, please add your Model to the modelzoo 
*************
-----------------x1-------------------
[tensor([1.])]
tensor([-0.9649])
-----------------ord_bin1-------------------
[tensor([1.])]
tensor([[0., 1.]])
-----------------ord_multi-------------------
[tensor([1.]), tensor([0.1714]), tensor([[0., 1.]])]
tensor([[0., 1., 0., 0.]])
-----------------ord_bin2-------------------
[tensor([0.6566]), tensor([[0., 1.]]), tensor([[1., 0., 0., 0.]])]
tensor([[1., 0.]])


## new version of dataloasdr v5

In [10]:
class GenericDataset_v5(Dataset):
    def __init__(
        self,
        df,
        target_col,
        target_nodes=None,
        parents_dataype_dict=None,
        transformation_terms_in_h=None,
        return_intercept_shift=True,
        transform=None
    ):
        """
        df: pd.DataFrame
        target_col: str
        target_nodes: dict mapping each node → metadata (including 'data_type')
        parents_dataype_dict: dict var_name → "cont"|"ord"|"other"
        transformation_terms_in_h: dict for intercept logic
        return_intercept_shift: whether to return (int_input, shift_list) or raw features
        transform: torchvision transform for images
        """
        self.return_intercept_shift = return_intercept_shift
        self.df = df.reset_index(drop=True)
        self.target_col = target_col
        self.target_nodes = target_nodes or {}
        self.parents_dataype_dict = parents_dataype_dict or {}
        self.predictors = list(self.parents_dataype_dict.keys())
        self.transform = transform
        self.transformation_terms_preprocessing = list((transformation_terms_in_h or {}).values())

        self.target_is_source = (self.target_nodes[self.target_col].get('node_type', '').lower() == "source")
        
        # do we need an explicit simple intercept?
        self.h_needs_simple_intercept = all('i' not in str(v) for v in self.transformation_terms_preprocessing)
        # count ordinal classes
        self.ordinal_num_classes = {
            v: self.df[v].nunique()
            for v in self.predictors
            if "ordinal" in self.parents_dataype_dict[v].lower()
            and "xn" in self.parents_dataype_dict[v].lower()
        }

        self.target_data_type = self.target_nodes[self.target_col].get('data_type', '').lower()
        self.target_num_classes = self.target_nodes[self.target_col].get('levels')

        # figure out the intercept and shift terms for preprocessed 
        if return_intercept_shift:
            self._set_intercept_shift_indexes()

        # checks
        self._check_multiclass_predictors_of_df()
        self._check_ordinal_levels()

    def _set_intercept_shift_indexes(self):
        # always inject a simple intercept term if no 'ci' present
        if not any('ci' in t for t in self.transformation_terms_preprocessing):
            self.transformation_terms_preprocessing.insert(0, 'si')

        self.intercept_indices = [
            i for i, term in enumerate(self.transformation_terms_preprocessing)
            if term.startswith(('si', 'ci'))
        ]

        self.shift_groups_indices = []
        current_key = None
        for i, term in enumerate(self.transformation_terms_preprocessing):
            if term.startswith(('cs', 'ls')):
                if len(term) > 2 and term[2].isdigit():
                    grp = term[:3]
                    if not self.shift_groups_indices or current_key != grp:
                        self.shift_groups_indices.append([i])
                        current_key = grp
                    else:
                        self.shift_groups_indices[-1].append(i)
                else:
                    self.shift_groups_indices.append([i])
                    current_key = None

    def _preprocess_inputs(self, x):
        """
        x: List of tensors, each with a dummy batch‑dim at 0
        Returns:
            int_inputs: (B, sum(intercept_dims))
            shift_list: list of (B, sum(dims_per_group)) or None
        """
        # intercept
        its = [x[i] for i in self.intercept_indices]
        if not its:
            raise ValueError("No intercept tensors found!")
        int_inputs = torch.cat([t.view(t.shape[0], -1) for t in its], dim=1)

        # shift‐groups
        shifts = []
        for grp in self.shift_groups_indices:
            parts = [x[i].view(x[i].shape[0], -1) for i in grp]
            shifts.append(torch.cat(parts, dim=1))

        return int_inputs, (shifts if shifts else None)

    def _transform_y(self, row):
        if self.target_data_type == "continous" or "yc" in self.target_data_type:
            return torch.tensor(row[self.target_col], dtype=torch.float32)
        elif self.target_num_classes:
            yi = int(row[self.target_col])
            return F.one_hot(
                torch.tensor(yi, dtype=torch.long),
                num_classes=self.target_num_classes
            ).float().squeeze()
        else:
            raise ValueError(
                f"Cannot encode target '{self.target_col}': "
                f"{self.target_data_type}/{self.target_num_classes}"
            )

    def _check_multiclass_predictors_of_df(self):
        for v in self.predictors:
            dt = self.parents_dataype_dict[v].lower()
            if "ordinal" in dt and "xn" in dt:
                vals = set(self.df[v].dropna().unique())
                if vals != set(range(len(vals))):
                    raise ValueError(
                        f"Ordinal predictor '{v}' must be zero‑indexed; got {sorted(vals)}"
                    )

    def _check_ordinal_levels(self):
        ords = []
        if "ordinal" in self.target_nodes[self.target_col]['data_type']:
            ords.append(self.target_col)
        ords += [
            v for v in self.predictors
            if "ordinal" in self.parents_dataype_dict[v].lower()
            and "xn" in self.parents_dataype_dict[v].lower()
        ]
        for v in ords:
            lvl = self.target_nodes[v].get('levels')
            if lvl is None:
                raise ValueError(f"Ordinal '{v}' missing 'levels' metadata.")
            uniq = sorted(self.df[v].dropna().unique())
            if uniq != list(range(lvl)):
                raise ValueError(
                    f"Ordinal '{v}' values {uniq} != expected 0…{lvl-1}."
                )

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        x_data = []

        # simple intercept if needed
        if self.h_needs_simple_intercept:
            x_data.append(torch.tensor(1.0))

        # source‐only shortcut
        if self.target_is_source:
            y = self._transform_y(row)
            if not self.return_intercept_shift:
                return tuple(x_data), y

            # batchify & preprocess
            batched = [x.unsqueeze(0) for x in x_data]
            int_in, shifts = self._preprocess_inputs(batched)
            # **squeeze off** our dummy batch‐dim:
            int_in = int_in.squeeze(0)              # -> (n_intercept,)
            shifts = [] if shifts is None else [s.squeeze(0) for s in shifts]
            return (int_in, shifts), y

        # build predictors
        for var in self.predictors:
            dt = self.parents_dataype_dict[var].lower()
            if dt == "continous" or "xc" in dt:
                x_data.append(torch.tensor(row[var], dtype=torch.float32))
            elif "ordinal" in dt and "xn" in dt:
                c = self.ordinal_num_classes[var]
                o = int(row[var])
                x_data.append(
                    F.one_hot(torch.tensor(o, dtype=torch.long), num_classes=c).float().squeeze()
                )
            else:
                img = Image.open(row[var]).convert("RGB")
                if self.transform:
                    img = self.transform(img)
                x_data.append(img)

        y = self._transform_y(row)

        if not self.return_intercept_shift:
            return tuple(x_data), y

        
        batched = [x.unsqueeze(0) for x in x_data]
        int_in, shifts = self._preprocess_inputs(batched)
        int_in = int_in.squeeze(0)
        shifts = [] if shifts is None else [s.squeeze(0) for s in shifts]
        return (int_in, shifts), y

def get_dataloader_v5(node, target_nodes, train_df, val_df, batch_size=32,return_intercept_shift=False, verbose=False):
    
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    ordered_parents_dataype_dict, ordered_transformation_terms_in_h, _ = ordered_parents(node, target_nodes)
    if verbose:
        print(f"Parents dtype: {ordered_parents_dataype_dict}")
    train_ds = GenericDataset_v5(train_df,target_col=node,target_nodes=target_nodes,parents_dataype_dict=ordered_parents_dataype_dict,transform=transform,transformation_terms_in_h=ordered_transformation_terms_in_h,return_intercept_shift=return_intercept_shift)
    val_ds = GenericDataset_v5(val_df,target_col=node,target_nodes=target_nodes,parents_dataype_dict=ordered_parents_dataype_dict,transform=transform,transformation_terms_in_h=ordered_transformation_terms_in_h,return_intercept_shift=return_intercept_shift)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    return train_loader, val_loader

In [82]:
data_type={key:value for key, value in zip(train_df.columns, ['continous']+['ordinal_Xn_Yo']+['ordinal_Xn_Yo']+['ordinal_Xn_Yo'])}
print(data_type)
configuration_dict=new_conf_dict(experiment_name,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)

adj_matrix=np.array([['0', '0', 'cs11', 'cs11'],
                     ['0', '0', 'cs12', 'cs12'],
                     ['0', '0', '0', 'cs'],
                     ['0', '0', '0', '0']])

return_intercept_shift=True


nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
levels_dict=create_levels_dict(df,data_type)
target_nodes=create_node_dict_v3(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict)

for node in target_nodes:
    print(f'-----------------{node}-------------------')
    train_loader, _ = get_dataloader_v5(node, target_nodes, train_df, val_df, batch_size=1,return_intercept_shift=return_intercept_shift, verbose=False)
    
    if return_intercept_shift:
        (int_input, shift_list), y =next(iter(train_loader))
        print(f'int_input {int_input}')
        print(f'shift_list {shift_list}')

        print(int_input.shape)
    else:
        x, y =next(iter(train_loader))
        print(x)
        print(y)
        


{'x1': 'continous', 'ord_bin1': 'ordinal_Xn_Yo', 'ord_multi': 'ordinal_Xn_Yo', 'ord_bin2': 'ordinal_Xn_Yo'}
*************
 Model has Complex intercepts and Complex shifts, please add your Model to the modelzoo 
*************
-----------------x1-------------------


int_input tensor([[1.]])
shift_list []
torch.Size([1, 1])
-----------------ord_bin1-------------------
int_input tensor([[1.]])
shift_list []
torch.Size([1, 1])
-----------------ord_multi-------------------
int_input tensor([[1.]])
shift_list [tensor([[1.0736, 0.0000, 1.0000]])]
torch.Size([1, 1])
-----------------ord_bin2-------------------
int_input tensor([[1.]])
shift_list [tensor([[0., 1., 0., 0.]]), tensor([[-0.7554,  1.0000,  0.0000]])]
torch.Size([1, 1])


In [15]:
data_type={key:value for key, value in zip(train_df.columns, ['continous']+['ordinal_Xn_Yc']+['ordinal_Xc_Yo']+['ordinal_Xn_Yc'])}
print(data_type)
configuration_dict=new_conf_dict(experiment_name,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)

adj_matrix=np.array([['0', 'cs', 'cs', 'ls'],
                     ['0', '0', 'ls', 'ci'],
                     ['0', '0', '0', 'cs'],
                     ['0', '0', '0', '0']])

return_intercept_shift=True
      
nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
levels_dict=create_levels_dict(df,data_type)
target_nodes=create_node_dict_v3(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict)

for node in target_nodes:
    print(f'-----------------{node}-------------------')
    train_loader, _ = get_dataloader_v5(node, target_nodes, train_df, val_df, batch_size=1,return_intercept_shift=return_intercept_shift, verbose=False)
    
    if return_intercept_shift:
        (int_input, shift_list), y =next(iter(train_loader))
        print(f'int_input {int_input}')
        print(f'shift_list {shift_list}')

        print(int_input.shape)
    else:
        x, y =next(iter(train_loader))
        print(x)
        print(y)
        

{'x1': 'continous', 'ord_bin1': 'ordinal_Xn_Yc', 'ord_multi': 'ordinal_Xc_Yo', 'ord_bin2': 'ordinal_Xn_Yc'}
*************
 Model has Complex intercepts and Complex shifts, please add your Model to the modelzoo 
*************
-----------------x1-------------------


int_input tensor([[1.]])
shift_list []
torch.Size([1, 1])
-----------------ord_bin1-------------------
int_input tensor([[1.]])
shift_list [tensor([[-0.5663]])]
torch.Size([1, 1])
-----------------ord_multi-------------------
int_input tensor([[1.]])
shift_list [tensor([[0.9686]]), tensor([[1., 0.]])]
torch.Size([1, 1])
-----------------ord_bin2-------------------
int_input tensor([[0., 1.]])
shift_list [tensor([[3.]]), tensor([[0.1958]])]
torch.Size([1, 2])


In [ ]:
def get_fully_specified_tram_model_v5(node: str, target_nodes: dict, verbose=True):
    """
    returns a Trammodel fully specified , according to CI groups and CS groups , for ordinal outcome and inputs

    """
    # Helper to detect ordinal with 'yo'
    def is_ordinal_yo(data_type: str) -> bool:
        return 'ordinal' in data_type and 'yo' in data_type.lower()

    # Determine number of thetas for ordinal nodes
    def compute_n_thetas(node_meta: dict):
        return node_meta['levels'] - 1 if is_ordinal_yo(node_meta['data_type']) else None

    # Compute number of input features for a given feature set
    def compute_n_features(feats, parents_datatype):
        n_features = 0
        for parent_name, _ in feats:
            parent_dt = parents_datatype[parent_name]
            if 'xn' in parent_dt.lower():
                n_features += target_nodes[parent_name]['levels']
            else:
                n_features += 1
        return n_features

    # Gather transformation terms and model class names
    _, terms_dict, model_names_dict = ordered_parents(node, target_nodes)
    model_dict = merge_transformation_dicts(terms_dict, model_names_dict)

    # Split into intercept and shift components by h_term prefixes
    intercepts_dict = {
        k: v for k, v in model_dict.items()
        if any(pref in v['h_term'] for pref in ("ci", "si"))
    }
    shifts_dict = {
        k: v for k, v in model_dict.items()
        if not any(pref in v['h_term'] for pref in ("ci", "si"))
    }

    # Build intercept network
    intercept_groups = group_by_base(intercepts_dict, prefixes=("ci", "si"))
    if intercept_groups:
        if len(intercept_groups) > 1:
            raise ValueError("Multiple complex intercept groups detected; only one is supported.")
        feats = next(iter(intercept_groups.values()))
        cls_name = feats[0][1]['class_name']
        base_cls = get_base_model_class(cls_name)
        # Use specified thetas or default for continuous
        number_thetas = compute_n_thetas(target_nodes[node]) or 20
        n_features = compute_n_features(feats, target_nodes[node]['parents_datatype'])
        nn_int = globals()[base_cls](n_features=n_features, n_thetas=number_thetas)
    else:
        theta_count = compute_n_thetas(target_nodes[node])
        nn_int = SimpleIntercept(n_thetas=theta_count) if theta_count is not None else SimpleIntercept()

    # Build shift networks
    shift_groups = group_by_base(shifts_dict, prefixes=("cs", "ls"))
    nn_shifts = []
    for feats in shift_groups.values():
        cls_name = feats[0][1]['class_name']
        base_cls = get_base_model_class(cls_name)
        n_features = compute_n_features(feats, target_nodes[node]['parents_datatype'])
        nn_shifts.append(globals()[base_cls](n_features=n_features))

    # Combine into final TramModel
    tram_model = TramModel(nn_int, nn_shifts)
    return tram_model

In [85]:
get_fully_specified_tram_model_v5(node, target_nodes, verbose=True)

TramModel(
  (nn_int): SimpleIntercept(
    (fc): Linear(in_features=1, out_features=1, bias=False)
  )
  (nn_shift): ModuleList(
    (0): ComplexShiftDefaultTabular(
      (fc1): Linear(in_features=4, out_features=64, bias=True)
      (relu1): ReLU()
      (fc2): Linear(in_features=64, out_features=128, bias=True)
      (relu2): ReLU()
      (fc3): Linear(in_features=128, out_features=64, bias=True)
      (relu3): ReLU()
      (fc4): Linear(in_features=64, out_features=1, bias=False)
    )
    (1): ComplexShiftDefaultTabular(
      (fc1): Linear(in_features=3, out_features=64, bias=True)
      (relu1): ReLU()
      (fc2): Linear(in_features=64, out_features=128, bias=True)
      (relu2): ReLU()
      (fc3): Linear(in_features=128, out_features=64, bias=True)
      (relu3): ReLU()
      (fc4): Linear(in_features=64, out_features=1, bias=False)
    )
  )
)

## TEST DATALAODER V5 and MODEL loader V5 together

In [92]:
def train_val_loop_v5(
    node,
    target_nodes,
    NODE_DIR,
    tram_model: torch.nn.Module,
    train_loader: torch.utils.data.DataLoader,
    val_loader: torch.utils.data.DataLoader,
    epochs: int,
    optimizer: torch.optim.Optimizer,
    use_scheduler: bool,
    scheduler: torch.optim.lr_scheduler._LRScheduler = None,
    save_linear_shifts: bool = False,
    verbose: int = 1,
    device: str = 'cpu'
):
    # Resolve device
    device = torch.device(device)
    # Paths for saving
    MODEL_PATH, LAST_MODEL_PATH, TRAIN_HIST_PATH, VAL_HIST_PATH = \
        model_train_val_paths(NODE_DIR)

    # Move model (and any existing params) onto device immediately
    tram_model = tram_model.to(device)

    # Prepare min/max tensors for contram scaling
    min_vals = torch.tensor(target_nodes[node]['min'], dtype=torch.float32, device=device)
    max_vals = torch.tensor(target_nodes[node]['max'], dtype=torch.float32, device=device)
    min_max = torch.stack([min_vals, max_vals], dim=0)

    # Load old training state if it exists
    if os.path.exists(MODEL_PATH) and os.path.exists(TRAIN_HIST_PATH) and os.path.exists(VAL_HIST_PATH):
        if verbose:
            print("Existing model found. Loading weights and history...")
        tram_model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
        with open(TRAIN_HIST_PATH, 'r') as f:
            train_loss_hist = json.load(f)
        with open(VAL_HIST_PATH, 'r') as f:
            val_loss_hist = json.load(f)
        start_epoch = len(train_loss_hist)
        best_val_loss = min(val_loss_hist)
    else:
        if verbose:
            print("No existing model found. Starting fresh...")
        train_loss_hist, val_loss_hist = [], []
        start_epoch = 0
        best_val_loss = float('inf')

    # Main loop
    for epoch in range(start_epoch, epochs):
        epoch_start = time.time()

        # --- Training ---
        tram_model.train()
        train_loss = 0.0
        train_start = time.time()
        for (int_input, shift_list), y in train_loader:
            # Move everything to device
            int_input = int_input.to(device)
            shift_list = [s.to(device) for s in shift_list]
            y = y.to(device)

            optimizer.zero_grad()
            y_pred = tram_model(int_input=int_input, shift_input=shift_list)

            if 'yo' in target_nodes[node]['data_type'].lower():
                loss = ontram_nll(y_pred, y)
            else:
                loss = contram_nll(y_pred, y, min_max=min_max)

            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        if use_scheduler and scheduler is not None:
            scheduler.step()

        avg_train_loss = train_loss / len(train_loader)
        train_loss_hist.append(avg_train_loss)
        train_time = time.time() - train_start

        # --- Validation ---
        tram_model.eval()
        val_loss = 0.0
        val_start = time.time()
        with torch.no_grad():
            for (int_input, shift_list), y in val_loader:
                int_input = int_input.to(device)
                shift_list = [s.to(device) for s in shift_list]
                y = y.to(device)

                y_pred = tram_model(int_input=int_input, shift_input=shift_list)
                if 'yo' in target_nodes[node]['data_type'].lower():
                    loss = ontram_nll(y_pred, y)
                else:
                    loss = contram_nll(y_pred, y, min_max=min_max)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        val_loss_hist.append(avg_val_loss)
        val_time = time.time() - val_start

        # --- Save linear shifts if requested ---
        if save_linear_shifts and hasattr(tram_model, 'nn_shift') and tram_model.nn_shift:
            shift_path = os.path.join(NODE_DIR, "linear_shifts_all_epochs.json")
            if os.path.exists(shift_path):
                with open(shift_path, 'r') as f:
                    all_shift_weights = json.load(f)
            else:
                all_shift_weights = {}

            epoch_weights = {}
            for i, shift_layer in enumerate(tram_model.nn_shift):
                if hasattr(shift_layer, 'fc') and hasattr(shift_layer.fc, 'weight'):
                    epoch_weights[f"shift_{i}"] = shift_layer.fc.weight.detach().cpu().tolist()
                else:
                    if verbose > 1:
                        print(f"shift_{i}: missing 'fc.weight'")
            all_shift_weights[f"epoch_{epoch+1}"] = epoch_weights
            with open(shift_path, 'w') as f:
                json.dump(all_shift_weights, f)
            if verbose > 1:
                print(f"Appended linear shift weights for epoch {epoch+1}")

        # --- Checkpointing ---
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(tram_model.state_dict(), MODEL_PATH)
            if verbose:
                print("Saved new best model.")
        # Always save last
        torch.save(tram_model.state_dict(), LAST_MODEL_PATH)
        with open(TRAIN_HIST_PATH, 'w') as f:
            json.dump(train_loss_hist, f)
        with open(VAL_HIST_PATH, 'w') as f:
            json.dump(val_loss_hist, f)

        # --- Epoch summary ---
        if verbose:
            total_time = time.time() - epoch_start
            print(
                f"Epoch {epoch+1}/{epochs}  "
                f"Train Loss: {avg_train_loss:.4f}  Val Loss: {avg_val_loss:.4f}  "
                f"[Train: {train_time:.2f}s  Val: {val_time:.2f}s  Total: {total_time:.2f}s]"
            )

In [93]:
DEV_TRAINING=True
train_list=['x1','x2','x3']#['x2']#'x1','x2']#,'x3']#['x1']#['x1','x2','x3']#,#,['x1','x2','x3'] # <-  set the nodes which have to be trained , useful if further training is required else lsit all vars

batch_size = 512#4112
epochs = 10# <- if you want a higher numbe rof epochs, set the number higher and it loads the old model and starts from there
learning_rate=0.01
use_scheduler =  False

In [94]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Train with GPU support.")
else:
    device = torch.device('cpu')
    print("No GPU found, train with CPU support.")

data_type={key:value for key, value in zip(train_df.columns, ['continous']+['ordinal_Xn_Yo']+['ordinal_Xn_Yo']+['ordinal_Xn_Yo'])}
print(data_type)
configuration_dict=new_conf_dict(experiment_name,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)

adj_matrix=np.array([['0', '0', 'cs11', 'cs11'],
                     ['0', '0', 'cs12', 'cs12'],
                     ['0', '0', '0', 'cs'],
                     ['0', '0', '0', '0']])

return_intercept_shift=True


nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
levels_dict=create_levels_dict(df,data_type)
target_nodes=create_node_dict_v3(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict)


for node in target_nodes:
    print(f'-----------------{node}-------------------')
    
    tram_model= get_fully_specified_tram_model_v5(node, target_nodes, verbose=True)
    train_loader, val_loader = get_dataloader_v5(node, target_nodes, train_df, val_df, batch_size=1,return_intercept_shift=return_intercept_shift, verbose=False)
    
    NODE_DIR = os.path.join(EXPERIMENT_DIR, f'{node}')
    os.makedirs(NODE_DIR, exist_ok=True)
    
    optimizer =torch.optim.Adam(tram_model.parameters(), lr=learning_rate)
    
    if use_scheduler:
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    else:
        scheduler = None
    
    
    if return_intercept_shift:
        (int_input, shift_list), y =next(iter(train_loader))
        train_val_loop_v5(
                    node,
                    target_nodes,
                    NODE_DIR,
                    tram_model,
                    train_loader,
                    val_loader,
                    epochs,
                    optimizer,
                    use_scheduler,
                    scheduler,
                    save_linear_shifts=False,
                    verbose=1,
                    device=device)


Train with GPU support.
{'x1': 'continous', 'ord_bin1': 'ordinal_Xn_Yo', 'ord_multi': 'ordinal_Xn_Yo', 'ord_bin2': 'ordinal_Xn_Yo'}
*************
 Model has Complex intercepts and Complex shifts, please add your Model to the modelzoo 
*************
-----------------x1-------------------
No existing model found. Starting fresh...
Saved new best model.
Epoch 1/10  Train Loss: 2.0808  Val Loss: 1.7482  [Train: 9.54s  Val: 0.90s  Total: 10.44s]
Saved new best model.
Epoch 2/10  Train Loss: 1.6717  Val Loss: 1.5816  [Train: 8.98s  Val: 0.92s  Total: 9.91s]
Saved new best model.
Epoch 3/10  Train Loss: 1.5658  Val Loss: 1.5086  [Train: 8.83s  Val: 0.89s  Total: 9.72s]
Saved new best model.
Epoch 4/10  Train Loss: 1.5104  Val Loss: 1.4644  [Train: 8.96s  Val: 0.92s  Total: 9.89s]
Saved new best model.
Epoch 5/10  Train Loss: 1.4748  Val Loss: 1.4354  [Train: 8.77s  Val: 0.89s  Total: 9.67s]
Saved new best model.
Epoch 6/10  Train Loss: 1.4500  Val Loss: 1.4154  [Train: 8.84s  Val: 0.91s  Tota